In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1749011983129_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# 1. Leer los datos desde S3 (asumiendo CSV)
df = spark.read.option("header", "true").csv("s3://final-gizmo/headlines/final/periodico=*/year=*/month=*/day=*/*.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+--------------------+
|     fecha|           categoria|             titular|              enlace|
+----------+--------------------+--------------------+--------------------+
|2025-06-03|Unidad investigativa|Ubicación secreta...|https://www.eltie...|
|2025-06-03|Unidad investigativa|Procuraduría abre...|https://www.eltie...|
|2025-06-03|    Justicia/Delitos|Fiscalía imputará...|https://www.eltie...|
|2025-06-03| Mundo/Eeuu y canada|Estados Unidos du...|https://www.eltie...|
|2025-06-03|Colombia/Otras ci...|Mamá de Tatiana H...|https://www.eltie...|
|2025-06-03|Politica/Partidos...|¿Claudia López, c...|https://www.eltie...|
|2025-06-03|Unidad investigativa|Los señalamientos...|https://www.eltie...|
|2025-06-03|              Bogota|La hipótesis sobr...|https://www.eltie...|
|2025-06-03|Unidad investigativa|Piden a firma de ...|https://www.eltie...|
|2025-06-03|        Mundo/Europa|Drones, submarino...|https://www.eltie...|
|2025-06-03|

In [4]:
# 2. Preprocesamiento básico
from pyspark.sql.functions import col, lower, regexp_replace

df_clean = df.select(
    col("categoria").alias("label"),
    lower(regexp_replace(col("titular"), "[^a-zA-ZáéíóúñÁÉÍÓÚÑ ]", "")).alias("clean_text")
).na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# 3. Transformaciones de texto (TF-IDF)
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="clean_text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
indexer = StringIndexer(inputCol="label", outputCol="labelIndex")

lr = LogisticRegression(featuresCol="features", labelCol="labelIndex", maxIter=10)

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, indexer, lr])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# 4. Entrenamiento del modelo
model = pipeline.fit(df_clean)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# 5. Aplicar modelo y ver resultados
predictions = model.transform(df_clean)
predictions.select("clean_text", "prediction", "probability").show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|clean_text                                    

In [15]:
# 6. Guardar resultados en S3 (como CSV o Parquet)
predictions.select("clean_text", "label", "prediction")\
    .write.mode("overwrite").csv("s3://final-gizmo/resultados/predicciones.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.sql.functions import col

predictions.withColumn("probability_str", col("probability").cast("string")) \
    .select("clean_text", "label", "prediction", "probability_str") \
    .write.mode("overwrite").csv("s3://final-gizmo/resultados/predicciones_prob.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
predictions.write.mode("overwrite").parquet("s3://final-gizmo/resultados/predicciones_parquet/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…